In [3]:
#!pip install mlflow

In [18]:
#!pip install google-cloud

In [32]:
#!pip install google-cloud-storage

In [45]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import mlflow

In [46]:
TRACKING_URI = "http://34.173.115.10:"

In [47]:
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment("sklearn-iris-prediction-gcp")

2022/12/21 14:09:03 INFO mlflow.tracking.fluent: Experiment with name 'sklearn-iris-prediction-gcp' does not exist. Creating a new experiment.


<Experiment: artifact_location='gs://aiteam-mlflow-storage-bucket/models/2', creation_time=1671608338614, experiment_id='2', last_update_time=1671608338614, lifecycle_stage='active', name='sklearn-iris-prediction-gcp', tags={}>

In [48]:
df = pd.read_csv("iris.csv")
print(df.size)
df.head()


750


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [49]:
train_df = df.sample(n=100)
train_df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
18,5.7,3.8,1.7,0.3,Setosa
57,4.9,2.4,3.3,1.0,Versicolor
146,6.3,2.5,5.0,1.9,Virginica
120,6.9,3.2,5.7,2.3,Virginica
130,7.4,2.8,6.1,1.9,Virginica


In [50]:
test_df = df.sample(n=150)
test_df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
67,5.8,2.7,4.1,1.0,Versicolor
135,7.7,3.0,6.1,2.3,Virginica
33,5.5,4.2,1.4,0.2,Setosa
36,5.5,3.5,1.3,0.2,Setosa
65,6.7,3.1,4.4,1.4,Versicolor


In [51]:
x_train,y_train = train_df.iloc[:, :-1], train_df.iloc[:, -1]
x_test,y_test   = test_df.iloc[:, :-1], test_df.iloc[:, -1]
print("X train : ",x_train.shape)
print("x test  : ",x_test.shape)

X train :  (100, 4)
x test  :  (150, 4)


In [52]:
flow_dict = {"Virginica":0,
"Versicolor":1,
"Setosa":2}

In [53]:
y_train = [flow_dict[x] for x in y_train]
y_test = [flow_dict[x] for x in y_test]

In [54]:
from sklearn.naive_bayes import GaussianNB
gnb = LogisticRegression()
gnb.fit(x_train,y_train)

LogisticRegression()

In [55]:

y_pred = gnb.predict(x_test)
accuracy = np.sum(y_test == y_pred) / len(y_test)
print("Model Accuracy: ",accuracy)

Model Accuracy:  0.9666666666666667


In [59]:
with mlflow.start_run():
    
    mlflow.set_tag("developer","tharhtet")
    mlflow.set_tag("version","prerelease")
    
    mlflow.log_param("traig-path","iris.csv")
    mlflow.log_param("train-amount","100")
    mlflow.log_param("val-amount","150")
    mlflow.sklearn.autolog()
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    gnb = LogisticRegression()
    gnb.fit(x_train,y_train)    
    y_pred = gnb.predict(x_test)
    accuracy = np.sum(y_test == y_pred) / len(y_test)

    filename = 'finalized_model.sav'
    pickle.dump(gnb, open(filename, 'wb'))
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_artifact(local_path=filename, artifact_path="models_pickle")
    #mlflow.sklearn.log_model(gnb, "iris-prediction", signature='test')

2022/12/21 14:15:47 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/12/21 14:15:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\tharh\.conda\envs\tharhtet\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."
